# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [21]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta, stats
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [11]:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [12]:
print(salaries.head())

                  Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary            NaN   
4              OEMC                 P           Hourly           20.0   

   Annual Salary  Hourly Rate  
0       101442.0          NaN  
1        94122.0          NaN  
2       101592.0          NaN  
3       110064.0          NaN  
4   

In [14]:
def convert_to_hourly(salary, salary_type):
    if salary_type == 'Salary':
         return salary / (40 * 52)
    elif salary_type == 'Hourly':
        return salary
    else:
        return None  # O manejar otros casos si hay otros tipos de salario


salaries['Hourly Rate'] = salaries.apply(lambda row: convert_to_hourly(row['Annual Salary'], row['Salary or Hourly']), axis=1)

salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,48.770192
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,45.250962
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,48.842308
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,52.915385
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,NaN


In [17]:
salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Hourly Rate'] = salaries['Typical Hours']
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,48.770192
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,45.250962
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,48.842308
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,52.915385
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,20.000000


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [19]:
import pandas as pd
from scipy.stats import ttest_1samp


#filtramos solo por hora
hourly_salaries = salaries[salaries['Salary or Hourly'] == 'Hourly']

hourly_wages = hourly_salaries['Hourly Rate'].dropna().astype(float)

# prueba t de una muestra solo
t_statistic, p_value = ttest_1samp(hourly_wages, 30)


print(t_statistic)
print(p_value)

#interpreatcion
alpha = 0.05
if p_value < alpha:
    print("El salario por hora es significativamente diferente de $30/hora.")
else:
    print("El salario por hora no es significativamente diferente de $30/hora.")


43.63629396989955
0.0
El salario por hora es significativamente diferente de $30/hora.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [34]:
#media y error sem
mean_hourly_wage = np.mean(hourly_wages)
sem_hourly_wage = stats.sem(hourly_wages)  # Error estándar de la media

#el intervalo de confianza del 95%
confidence_interval = stats.t.interval(0.95, len(hourly_wages)-1, loc=mean_hourly_wage, scale=sem_hourly_wage)

# Mostrar los resultados
print(f"Media: {mean_hourly_wage}")
print(f"Error estándar: {sem_hourly_wage}")
print(f"Intervalo de confianza del 95%: {confidence_interval}")



Media: 32.78855771628024
Error estándar: 0.1352368565101596
Intervalo de confianza del 95%: (32.52345834488425, 33.05365708767623)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [30]:
from statsmodels.stats.proportion import proportions_ztest

#z test:  El estadístico Z es una medida estadística que indica cuántos desviaciones estándar está un valor particular por encima o por debajo de la media en un conjunto de datos normalizados.

#La fórmula para calcular el estadístico Z de un valor X en una distribución normal con media μ y desviación estándar σ es:
#Z=(X−μ)/σ

#trabajadores por hora
hourly_count = salaries[salaries['Salary or Hourly'] == 'Hourly'].shape[0]

# tamaño total de la muestra
total_count = salaries.shape[0]

# proporción esperada
proportion_expected = 0.25 #que podria ser distinta

#prueba de proporciones
count = hourly_count
nobs = total_count
z_statistic, p_value = proportions_ztest(count, nobs, value=proportion_expected)

print(f"Estadístico z: {z_statistic}")
print(f"Valor p: {p_value}")


alpha = 0.05
if p_value < alpha:
    print("La proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("La proporción de trabajadores por hora no es significativamente diferente del 25%.")



Estadístico z: -3.5099964213703005
Valor p: 0.0004481127249057967
La proporción de trabajadores por hora es significativamente diferente del 25%.


1.13.0
